<p><font size="6" color='grey'> <b>

Generative KI. Verstehen. Anwenden. Gestalten.
</b></font> </br></p>

<p><font size="5" color='grey'> <b> Chat Memory Patterns</b></font> </br></p>

---

In [1]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

✓ OPENAI_API_KEY erfolgreich gesetzt

Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]

Installierte LangChain- und LangGraph-Bibliotheken:
langchain                                1.1.0
langchain-chroma                         1.0.0
langchain-classic                        1.0.0
langchain-community                      0.4.1
langchain-core                           1.1.0
langchain-ollama                         1.0.0
langchain-openai                         1.1.0
langchain-text-splitters                 1.0.0
langgraph                                1.0.3
langgraph-checkpoint                     3.0.1
langgraph-prebuilt                       1.0.5
langgraph-sdk                            0.2.10

IP-Adresse: 34.147.124.34
Hostname: 34.124.147.34.bc.googleusercontent.com
Stadt: Groningen
Region: Groningen
Land: NL
Koordinaten: 53.2192,6.5667
Provider: AS396982 Google LLC
Postleitzahl: 9711
Zeitzone: Europe/Amsterdam


# 1 | Intro
---

<p><font color='black' size="5">
Zustandslosigkeit von LLMs
</font></p>

Large Language Models (LLMs) wie GPT sind von Natur aus **zustandslos** - sie verfugen uber kein eingebautes Gedachtnis. Jede Anfrage wird isoliert verarbeitet, ohne Bezug zu vorherigen Interaktionen. Deshalb muss der Chatverlauf (Historie) bei jeder Anfrage neu ubergeben werden.

```
Ohne Memory:
User: "Mein Name ist Max"
AI: "Hallo Max!"
User: "Wie heisse ich?"
AI: "Das habe ich nicht gespeichert."
```

**Dieses Notebook zeigt Memory-Patterns mit reinem Python (ohne LangGraph):**

| Pattern | Beschreibung | Anwendungsfall |
|---------|--------------|----------------|
| **Python-Liste** | Einfachste Losung | Prototyping, kurze Sessions |
| **Trimming** | Nur letzte N Nachrichten | Token-Limit einhalten |
| **Summary** | Alte Nachrichten zusammenfassen | Lange Sessions, Kontext erhalten |
| **Datenbank** | Persistente Speicherung | Production, Multi-User |

# 2 | Short-term Memory (Python-Liste)
---

Die einfachste Form von Memory: Eine **Python-Liste**, die alle Nachrichten speichert und bei jedem API-Call mitgesendet wird.

In [2]:
# Importe
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

# System-Prompt
system_prompt = "Du bist ein hilfreicher und humorvoller KI-Assistent."

# Prompt-Template mit Historie (MessagesPlaceholder nimmt die Historie entgegen)
prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{user_input}")
])

# LLM (Kurznotation: "provider:model")
llm = init_chat_model("openai:gpt-4o-mini", temperature=0)

# Parser
parser = StrOutputParser()

# Chain
chain = prompt | llm | parser

In [3]:
# Chat-Funktion mit manueller Historien-Verwaltung
def chat(chat_history: list, user_input: str) -> list:
    """Fuhrt eine Chat-Interaktion mit manueller Historien-Verwaltung durch."""

    # Chain aufrufen (Historie wird im Prompt mitgeschickt)
    response = chain.invoke({
        'system_prompt': system_prompt,
        'chat_history': chat_history,
        'user_input': user_input
    })

    # Ausgabe
    mprint(f"### Mensch:\n{user_input}")
    mprint(f"### KI:\n{response}\n")

    # Memory (Liste) MANUELL aktualisieren
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response))

    return chat_history

In [4]:
# Historie initialisieren
chat_history = [SystemMessage(content=system_prompt)]


# Konversation
chat_history = chat(chat_history, "Mein Name ist Max")
chat_history = chat(chat_history, "Ich mag Python-Programmierung")
chat_history = chat(chat_history, "Weisst du noch, wie ich heisse und was ich mag?")

mprint("### Gespeicherte Nachrichten (Liste):\n---")
for msg in chat_history:
    mprint(f"  **{msg.type}**:   {msg.content}")

### Mensch:
Mein Name ist Max

### KI:
Hallo Max! Schön, dich kennenzulernen! Wie kann ich dir heute helfen? Oder möchtest du einfach ein bisschen plaudern?


### Mensch:
Ich mag Python-Programmierung

### KI:
Das ist großartig, Max! Python ist eine fantastische Sprache – so vielseitig und benutzerfreundlich. Was machst du gerne mit Python? Programmierst du Spiele, Webanwendungen oder vielleicht etwas ganz anderes?


### Mensch:
Weisst du noch, wie ich heisse und was ich mag?

...

  **ai**:   Hallo Max! Schön, dich kennenzulernen! Wie kann ich dir heute helfen? Oder möchtest du einfach ein bisschen plaudern?

  **human**:   Ich mag Python-Programmierung

  **ai**:   Das ist großartig, Max! Python ist eine fantastische Sprache – so vielseitig und benutzerfreundlich. Was machst du gerne mit Python? Programmierst du Spiele, Webanwendungen oder vielleicht etwas ganz anderes?

  **human**:   Weisst du noch, wie ich heisse und was ich mag?

  **ai**:   Natürlich, Max! Du magst Python-Programmierung. Ich habe ein ausgezeichnetes Gedächtnis für solche Dinge – zumindest bis ich einen Systemneustart habe! 😄 Gibt es etwas Bestimmtes, worüber du in Bezug auf Python sprechen möchtest?

<p><font color='darkblue' size="4">
Problem:
</font></p>

- Keine automatische Session-Verwaltung (Multi-User)
- Manuelles Memory-Management fehleranfallig
- **Bei langen Konversationen: Token-Limit wird uberschritten!**

# 3 | Trimming (Sliding Window)
---

**Strategie:** Behalte nur die letzten *n* Nachrichten. Altere Nachrichten werden entfernt.

```
Vor Trimming (max=4):
[msg1, msg2, msg3, msg4, msg5, msg6] -> 6 Nachrichten

Nach Trimming:
[msg3, msg4, msg5, msg6] -> nur die letzten 4
```

**Vorteil:** Einfach, Token-Limit garantiert  
**Nachteil:** Fruhere Informationen gehen verloren

In [5]:
# Konfiguration
MAX_MESSAGES = 6  # Maximale Anzahl Nachrichten (Human + AI)

def trim_history(chat_history: list, max_messages: int = MAX_MESSAGES) -> list:
    """Behalt nur die letzten n Nachrichten."""
    if len(chat_history) > max_messages:
        trimmed = chat_history[-max_messages:]
        mprint(f"**Trimming:** {len(chat_history)} -> {len(trimmed)} Nachrichten")
        return trimmed
    return chat_history

In [6]:
def chat_with_trimming(chat_history: list, user_input: str, max_messages: int = MAX_MESSAGES) -> list:
    """Chat mit automatischem Trimming der Historie."""

    # Trimmen VOR dem API-Call
    trimmed_history = trim_history(chat_history, max_messages)

    # Chain aufrufen mit getrimmter Historie
    response = chain.invoke({
        'system_prompt': system_prompt,
        'chat_history': trimmed_history,
        'user_input': user_input
    })

    # Ausgabe
    mprint(f"### Mensch:\n{user_input}")
    mprint(f"### KI:\n{response}\n")

    # Zur ORIGINALEN Historie hinzufugen (nicht zur getrimmten!)
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response))

    return chat_history

In [7]:
# Demo: Trimming in Aktion
mprint("## Trimming Demo (max 6 Nachrichten)")
mprint("---")

history_trimmed = [SystemMessage(content=system_prompt)]


# Erste Nachrichten
history_trimmed = chat_with_trimming(history_trimmed, "Mein Name ist Max")
history_trimmed = chat_with_trimming(history_trimmed, "Ich wohne in Koln")
history_trimmed = chat_with_trimming(history_trimmed, "Ich mag Python")

# Jetzt uberschreiten wir das Limit (6 Nachrichten = 3 Frage-Antwort-Paare)
history_trimmed = chat_with_trimming(history_trimmed, "Ich habe eine Katze namens Neo")
history_trimmed = chat_with_trimming(history_trimmed, "Test-Nachricht 5")

# Diese Nachricht lost Trimming aus
history_trimmed = chat_with_trimming(history_trimmed, "Wie heisse ich?")  # Fruhe Info konnte verloren sein!

mprint(f"### Gespeicherte Nachrichten: {len(history_trimmed)}")

## Trimming Demo (max 6 Nachrichten)

---

### Mensch:
Mein Name ist Max

### KI:
Hallo Max! Schön, dich kennenzulernen! Wie kann ich dir heute helfen?


### Mensch:
Ich wohne in Koln

...

### KI:
Test-Nachricht empfangen! Wenn du noch mehr testen oder etwas anderes besprechen möchtest, lass es mich wissen! 🐾


**Trimming:** 10 -> 6 Nachrichten

### Mensch:
Wie heisse ich?

### KI:
Das ist eine gute Frage! Leider kann ich deinen Namen nicht wissen, es sei denn, du sagst es mir. Aber ich bin sicher, dass du einen tollen Namen hast! Wie darf ich dich nennen?


### Gespeicherte Nachrichten: 12

# 4 | Summary (Zusammenfassung)
---

**Strategie:** Statt alte Nachrichten zu loschen, werden sie **zusammengefasst**. Die Zusammenfassung ersetzt die alten Nachrichten.

```
Vor Summary:
[msg1, msg2, msg3, msg4, msg5, msg6, msg7, msg8] -> 8 Nachrichten

Nach Summary:
["Zusammenfassung: User heisst Max, mag Python...", msg7, msg8]
```

**Vorteil:** Wichtige Informationen bleiben erhalten  
**Nachteil:** Zusatzlicher LLM-Call fur Zusammenfassung

In [8]:
# Konfiguration
MAX_BEFORE_SUMMARY = 8   # Ab dieser Anzahl wird zusammengefasst
MESSAGES_TO_SUMMARIZE = 6  # So viele alte Nachrichten zusammenfassen
RECENT_TO_KEEP = 2        # So viele neueste Nachrichten behalten

# Zusammenfassungs-Prompt
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "Fasse die folgende Konversation in 2-3 Satzen zusammen. Behalte wichtige Fakten wie Namen, Orte und Praferenzen."),
    ("human", "{conversation}")
])

summary_chain = summary_prompt | llm | parser

In [9]:
def summarize_messages(messages: list) -> str:
    """Erstellt eine Zusammenfassung der Nachrichten."""
    conversation_text = "\n".join([
        f"{msg.type}: {msg.content}" for msg in messages
    ])

    summary = summary_chain.invoke({"conversation": conversation_text})
    return summary


def chat_with_summary(chat_history: list, user_input: str, summary_context: str = "") -> tuple:
    """Chat mit automatischer Zusammenfassung bei langer Historie."""

    # Prufen ob Zusammenfassung notig ist
    if len(chat_history) >= MAX_BEFORE_SUMMARY:
        mprint(f"**Summary:** Historie zu lang ({len(chat_history)} Nachrichten). Fasse zusammen...")

        # Alte Nachrichten zusammenfassen
        to_summarize = chat_history[:MESSAGES_TO_SUMMARIZE]
        new_summary = summarize_messages(to_summarize)

        # Bisherige Zusammenfassung + neue kombinieren
        if summary_context:
            summary_context = f"{summary_context}\n\nNeuere Zusammenfassung: {new_summary}"
        else:
            summary_context = new_summary

        mprint(f"**Neue Zusammenfassung:** {new_summary[:100]}...")

        # Historie kurzen (nur die neuesten behalten)
        chat_history = chat_history[-RECENT_TO_KEEP:]

    # System-Prompt mit Zusammenfassung erweitern
    enhanced_system = system_prompt
    if summary_context:
        enhanced_system = f"{system_prompt}\n\nBisheriger Kontext (Zusammenfassung): {summary_context}"

    # Chain aufrufen
    response = chain.invoke({
        'system_prompt': enhanced_system,
        'chat_history': chat_history,
        'user_input': user_input
    })

    # Ausgabe
    mprint(f"### Mensch:\n{user_input}")
    mprint(f"### KI:\n{response}\n")

    # Historie aktualisieren
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response))

    return chat_history, summary_context

In [10]:
# Demo: Summary in Aktion
mprint("## Summary Demo")
mprint("---")

history_summary = [SystemMessage(content=system_prompt)]

summary_ctx = ""

# Wichtige Informationen am Anfang
history_summary, summary_ctx = chat_with_summary(history_summary, "Mein Name ist Max", summary_ctx)
history_summary, summary_ctx = chat_with_summary(history_summary, "Ich wohne in Koln", summary_ctx)
history_summary, summary_ctx = chat_with_summary(history_summary, "Ich mag Python", summary_ctx)
history_summary, summary_ctx = chat_with_summary(history_summary, "Meine Katze heisst Neo", summary_ctx)

# Fullnachrichten um Summary auszulosen
for i in range(5):
    history_summary, summary_ctx = chat_with_summary(history_summary, f"Test {i+1}", summary_ctx)

# Nach Summary: Kann die KI sich noch an den Namen erinnern?
history_summary, summary_ctx = chat_with_summary(history_summary, "Wie heisse ich und wie heisst meine Katze?", summary_ctx)

mprint(f"### Aktuelle Historie: {len(history_summary)} Nachrichten")
mprint(f"### Gespeicherte Zusammenfassung:\n{summary_ctx}")

## Summary Demo

---

### Mensch:
Mein Name ist Max

### KI:
Hallo Max! Schön, dich kennenzulernen! Wie kann ich dir heute helfen?


### Mensch:
Ich wohne in Koln

...

### KI:
Test 5: Erfolgreich bestanden! 🎊 Du bist wirklich unaufhaltsam! Hast du vielleicht einen speziellen Test im Kopf, oder möchtest du einfach nur feiern? Vielleicht mit einem virtuellen Katzenleckerlies für Neo? 🐾😄


### Mensch:
Wie heisse ich und wie heisst meine Katze?

### KI:
Du heißt Max und deine Katze heißt Neo! 🐱✨ Wenn ich ein Gedächtnis hätte, würde ich sagen, du bist ein ganz schöner Glückspilz mit so einem coolen Namen für deine Katze! Was möchtest du als Nächstes wissen oder besprechen?


### Aktuelle Historie: 8 Nachrichten

### Gespeicherte Zusammenfassung:
Max wohnt in Köln und mag die Programmiersprache Python. Er schätzt die Vielseitigkeit von Python, die von Webentwicklung bis zu künstlicher Intelligenz reicht.

Neuere Zusammenfassung: Der Mensch hat eine Katze namens Neo, die möglicherweise geheimnisvoll und besonders ist. Es wurden zwei Tests durchgeführt, die beide erfolgreich waren, und die KI bietet an, weitere Fragen zu beantworten oder über die Katze zu plaudern.

# 5 | RunnableWithMessageHistory (LCEL)
---

`RunnableWithMessageHistory` ist die **offizielle LCEL-Alternative** zu den deprecated Memory-Klassen (`ConversationBufferMemory`, etc.). Es wrapped eine Chain und verwaltet die Chat-Historie automatisch.

**Vorteile gegenuber manueller Verwaltung:**
- Automatisches Laden/Speichern der Historie
- Session-Management uber `session_id`
- Kompatibel mit Streaming und Async

```python
from langchain_core.runnables.history import RunnableWithMessageHistory
```

In [11]:
# Importe fur RunnableWithMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# Session-Store (Dictionary: session_id -> ChatMessageHistory)
session_store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    """Gibt die Historie fur eine Session zuruck (oder erstellt eine neue)."""
    if session_id not in session_store:
        session_store[session_id] = InMemoryChatMessageHistory()
    return session_store[session_id]

In [12]:
# Prompt mit Historie-Platzhalter
prompt_with_history = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history"),  # Hier wird die Historie eingefugt
    ("human", "{input}")
])

# Basis-Chain (ohne Memory)
base_chain = prompt_with_history | llm | parser

# Chain MIT automatischem Memory-Management wrappen
chain_with_history = RunnableWithMessageHistory(
    runnable=base_chain,
    get_session_history=get_session_history,
    input_messages_key="input",      # Key fur neue User-Nachricht
    history_messages_key="history"   # Key fur die Historie im Prompt
)

print("Chain mit RunnableWithMessageHistory erstellt")

Chain mit RunnableWithMessageHistory erstellt


In [13]:
# Helper-Funktion fur Chat mit Session-ID
def chat_with_session(session_id: str, user_input: str) -> str:
    """Chattet mit automatischem Memory uber RunnableWithMessageHistory."""

    # Config mit Session-ID (PFLICHT!)
    config = {"configurable": {"session_id": session_id}}

    # Chain aufrufen - Historie wird automatisch geladen/gespeichert
    response = chain_with_history.invoke(
        {"input": user_input},
        config=config
    )

    mprint(f"**[{session_id}] Mensch:** {user_input}")
    mprint(f"**[{session_id}] KI:** {response}\n")

    return response

In [14]:
# Demo: RunnableWithMessageHistory
mprint("## RunnableWithMessageHistory Demo")
mprint("---")

# Session 1: Max
chat_with_session("max", "Hallo! Ich bin Max aus Munchen.")
chat_with_session("max", "Ich programmiere gerne in Python.")

# Session 2: Emma (separate Historie!)
chat_with_session("emma", "Hi! Ich bin Emma und mag Machine Learning.")

# Zuruck zu Max - Memory bleibt erhalten!
chat_with_session("max", "Woher komme ich und was ist mein Hobby?")

# Session-Store anzeigen
mprint("### Gespeicherte Sessions:")
for sid, history in session_store.items():
    mprint(f"- **{sid}**: {len(history.messages)} Nachrichten")

## RunnableWithMessageHistory Demo

---

**[max] Mensch:** Hallo! Ich bin Max aus Munchen.

**[max] KI:** Hallo Max! Schön, dich kennenzulernen! Wie läuft's in München? Gibt es etwas Bestimmtes, worüber du plaudern möchtest? 😊


**[max] Mensch:** Ich programmiere gerne in Python.

...

**[max] Mensch:** Woher komme ich und was ist mein Hobby?

**[max] KI:** Du kommst aus München und dein Hobby ist das Programmieren, insbesondere in Python! Wenn ich noch mehr über deine Projekte oder Interessen erfahren könnte, wäre ich ganz Ohr! Gibt es etwas Bestimmtes, das du in Python gerne machst? 😊


### Gespeicherte Sessions:

- **max**: 6 Nachrichten

- **emma**: 2 Nachrichten

In [15]:
# Historie einer Session anzeigen
def show_session_history(session_id: str):
    """Zeigt die Nachrichten einer Session."""
    if session_id not in session_store:
        print(f"Session '{session_id}' nicht gefunden")
        return

    history = session_store[session_id]
    mprint(f"### Historie: {session_id}")
    for i, msg in enumerate(history.messages, 1):
        role = "Human" if msg.type == "human" else "KI"
        mprint(f"{i}. **{role}:** {msg.content}")

show_session_history("max")

### Historie: max

1. **Human:** Hallo! Ich bin Max aus Munchen.

2. **KI:** Hallo Max! Schön, dich kennenzulernen! Wie läuft's in München? Gibt es etwas Bestimmtes, worüber du plaudern möchtest? 😊

3. **Human:** Ich programmiere gerne in Python.

4. **KI:** Das ist großartig! Python ist eine super vielseitige Sprache. Was programmierst du denn so? Hast du ein Lieblingsprojekt oder ein bestimmtes Thema, das dich interessiert? 🐍💻

5. **Human:** Woher komme ich und was ist mein Hobby?

6. **KI:** Du kommst aus München und dein Hobby ist das Programmieren, insbesondere in Python! Wenn ich noch mehr über deine Projekte oder Interessen erfahren könnte, wäre ich ganz Ohr! Gibt es etwas Bestimmtes, das du in Python gerne machst? 😊

<p><font color='darkblue' size="4">
Vergleich: RunnableWithMessageHistory vs. LangGraph
</font></p>

| Aspekt | RunnableWithMessageHistory | LangGraph |
|--------|---------------------------|-----------|
| **Komplexitat** | Einfach | Fortgeschritten |
| **Persistenz** | Manuell (Store) | Checkpointer |
| **RemoveMessage** | Nein | Ja |
| **Trimming** | Manuell | `trim_messages` |
| **Multi-Agent** | Nein | Ja |

**Empfehlung:** `RunnableWithMessageHistory` fur einfache Chains, LangGraph fur komplexe Agents.

# 6 | Long-term Memory (SQLite)
---

**Problem:** Beim Neustart der Anwendung geht die Historie verloren.

**Losung:** Persistente Speicherung in einer **SQLite-Datenbank**.

| Speicherart | Persistenz | Multi-User | Anwendung |
|-------------|------------|------------|------------|
| Python-Liste | Nein | Nein | Prototyping |
| RunnableWithMessageHistory | Nein* | Ja | Einfache Chains |
| SQLite | Ja | Ja | Lokale Apps |

*Mit FileChatMessageHistory moglich

In [16]:
import sqlite3
from datetime import datetime
from typing import List, Dict, Optional

DB_PATH = "./chat_memory.db"

In [17]:
class ChatMemoryDB:
    """Einfache Chat-Memory-Datenbank mit SQLite."""

    def __init__(self, db_path: str = DB_PATH):
        self.db_path = db_path
        self._init_db()

    def _init_db(self):
        """Erstellt die Tabellen falls nicht vorhanden."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
                CREATE TABLE IF NOT EXISTS messages (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    thread_id TEXT NOT NULL,
                    role TEXT NOT NULL,
                    content TEXT NOT NULL,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
                )
            """)
            conn.execute("""
                CREATE TABLE IF NOT EXISTS summaries (
                    thread_id TEXT PRIMARY KEY,
                    summary TEXT,
                    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
                )
            """)
            conn.execute("CREATE INDEX IF NOT EXISTS idx_thread ON messages(thread_id)")
            conn.commit()
        print(f"Datenbank initialisiert: {self.db_path}")

    def save_message(self, thread_id: str, role: str, content: str):
        """Speichert eine Nachricht."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute(
                "INSERT INTO messages (thread_id, role, content) VALUES (?, ?, ?)",
                (thread_id, role, content)
            )
            conn.commit()

    def get_history(self, thread_id: str, limit: Optional[int] = None) -> List[Dict]:
        """Ladt die Historie eines Threads."""
        with sqlite3.connect(self.db_path) as conn:
            if limit:
                rows = conn.execute(
                    "SELECT role, content FROM messages WHERE thread_id = ? ORDER BY id DESC LIMIT ?",
                    (thread_id, limit)
                ).fetchall()
                rows = list(reversed(rows))
            else:
                rows = conn.execute(
                    "SELECT role, content FROM messages WHERE thread_id = ? ORDER BY id",
                    (thread_id,)
                ).fetchall()
        return [{"role": r[0], "content": r[1]} for r in rows]

    def get_message_count(self, thread_id: str) -> int:
        """Zahlt die Nachrichten eines Threads."""
        with sqlite3.connect(self.db_path) as conn:
            count = conn.execute(
                "SELECT COUNT(*) FROM messages WHERE thread_id = ?",
                (thread_id,)
            ).fetchone()[0]
        return count

    def save_summary(self, thread_id: str, summary: str):
        """Speichert/aktualisiert die Zusammenfassung eines Threads."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
                INSERT OR REPLACE INTO summaries (thread_id, summary, updated_at)
                VALUES (?, ?, CURRENT_TIMESTAMP)
            """, (thread_id, summary))
            conn.commit()

    def get_summary(self, thread_id: str) -> Optional[str]:
        """Ladt die Zusammenfassung eines Threads."""
        with sqlite3.connect(self.db_path) as conn:
            row = conn.execute(
                "SELECT summary FROM summaries WHERE thread_id = ?",
                (thread_id,)
            ).fetchone()
        return row[0] if row else None

    def list_threads(self) -> List[Dict]:
        """Listet alle Threads mit Statistiken."""
        with sqlite3.connect(self.db_path) as conn:
            rows = conn.execute("""
                SELECT thread_id, COUNT(*) as msg_count, MAX(timestamp) as last_msg
                FROM messages
                GROUP BY thread_id
                ORDER BY last_msg DESC
            """).fetchall()
        return [{"thread_id": r[0], "messages": r[1], "last_activity": r[2]} for r in rows]

    def delete_thread(self, thread_id: str):
        """Loscht einen Thread."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("DELETE FROM messages WHERE thread_id = ?", (thread_id,))
            conn.execute("DELETE FROM summaries WHERE thread_id = ?", (thread_id,))
            conn.commit()

In [18]:
class PersistentChatbot:
    """Chatbot mit persistentem Memory uber SQLite."""

    def __init__(self, db_path: str = DB_PATH, max_context: int = 10):
        self.db = ChatMemoryDB(db_path)
        self.max_context = max_context
        self.llm = init_chat_model("openai:gpt-4o-mini", temperature=0.7)
        self.system_prompt = "Du bist ein hilfreicher KI-Assistent mit Gedachtnis."

    def _history_to_messages(self, history: List[Dict]) -> list:
        """Konvertiert DB-Historie zu LangChain-Messages."""
        messages = []
        for item in history:
            if item["role"] == "human":
                messages.append(HumanMessage(content=item["content"]))
            else:
                messages.append(AIMessage(content=item["content"]))
        return messages

    def chat(self, thread_id: str, user_input: str) -> str:
        """Sendet eine Nachricht und speichert die Antwort."""

        # Historie laden (mit Limit)
        history = self.db.get_history(thread_id, limit=self.max_context)
        summary = self.db.get_summary(thread_id)

        # System-Prompt mit Summary erweitern
        enhanced_system = self.system_prompt
        if summary:
            enhanced_system = f"{self.system_prompt}\n\nKontext aus fruheren Gesprachen: {summary}"

        # Messages zusammenbauen
        messages = [SystemMessage(content=enhanced_system)]
        messages.extend(self._history_to_messages(history))
        messages.append(HumanMessage(content=user_input))

        # LLM aufrufen
        response = self.llm.invoke(messages)
        response_text = response.content

        # In DB speichern
        self.db.save_message(thread_id, "human", user_input)
        self.db.save_message(thread_id, "ai", response_text)

        return response_text

    def show_history(self, thread_id: str):
        """Zeigt die Historie eines Threads."""
        history = self.db.get_history(thread_id)
        mprint(f"### Thread: {thread_id} ({len(history)} Nachrichten)")
        mprint("---")
        for i, msg in enumerate(history, 1):
            role = "Human" if msg["role"] == "human" else "KI"
            mprint(f"{i}. **{role}:** {msg['content']}")

    def list_threads(self):
        """Listet alle Threads."""
        threads = self.db.list_threads()
        mprint("### Alle Threads:")
        mprint("---")
        for t in threads:
            mprint(f"- **{t['thread_id']}**: {t['messages']} Nachrichten (zuletzt: {t['last_activity']})")

In [19]:
# Demo: Persistenter Chatbot
mprint("## SQLite-Chatbot Demo")
mprint("---")

bot = PersistentChatbot()

# Thread 1: Max
print("\n--- Thread: max_session ---")
response = bot.chat("max_session", "Hallo! Ich bin Max und komme aus Munchen.")
mprint(f"**KI:** {response}")

response = bot.chat("max_session", "Ich interessiere mich fur Machine Learning.")
mprint(f"**KI:** {response}")

# Thread 2: Emma
print("\n--- Thread: emma_session ---")
response = bot.chat("emma_session", "Hi! Ich bin Emma aus Berlin.")
mprint(f"**KI:** {response}")

# Zuruck zu Max - Memory bleibt erhalten!
print("\n--- Zuruck zu max_session ---")
response = bot.chat("max_session", "Woher komme ich nochmal?")
mprint(f"**KI:** {response}")

## SQLite-Chatbot Demo

---

Datenbank initialisiert: ./chat_memory.db

--- Thread: max_session ---


**KI:** Hallo Max! Schön, dich kennenzulernen. Wie kann ich dir heute helfen?

**KI:** Das ist großartig! Machine Learning ist ein spannendes und vielseitiges Feld. Gibt es einen bestimmten Bereich oder ein Thema im Machine Learning, das dich besonders interessiert? Zum Beispiel: Supervised Learning, Unsupervised Learning, neuronale Netze oder praktische Anwendungen?


--- Thread: emma_session ---


**KI:** Hallo Emma! Schön, dich kennenzulernen. Wie kann ich dir heute helfen?


--- Zuruck zu max_session ---


**KI:** Du hast mir gesagt, dass du aus München kommst. Gibt es etwas Bestimmtes über München oder Machine Learning, worüber du sprechen möchtest?

In [20]:
# Alle Threads anzeigen
bot.list_threads()

### Alle Threads:

---

- **max_session**: 6 Nachrichten (zuletzt: 2025-12-03 13:59:26)

- **emma_session**: 2 Nachrichten (zuletzt: 2025-12-03 13:59:25)

In [21]:
# Historie eines Threads anzeigen
bot.show_history("max_session")

### Thread: max_session (6 Nachrichten)

---

1. **Human:** Hallo! Ich bin Max und komme aus Munchen.

2. **KI:** Hallo Max! Schön, dich kennenzulernen. Wie kann ich dir heute helfen?

3. **Human:** Ich interessiere mich fur Machine Learning.

4. **KI:** Das ist großartig! Machine Learning ist ein spannendes und vielseitiges Feld. Gibt es einen bestimmten Bereich oder ein Thema im Machine Learning, das dich besonders interessiert? Zum Beispiel: Supervised Learning, Unsupervised Learning, neuronale Netze oder praktische Anwendungen?

5. **Human:** Woher komme ich nochmal?

6. **KI:** Du hast mir gesagt, dass du aus München kommst. Gibt es etwas Bestimmtes über München oder Machine Learning, worüber du sprechen möchtest?

<p><font color='darkblue' size="4">
Test: Neustart-Persistenz
</font></p>

Die Daten bleiben auch nach Neustart erhalten. Fuhren Sie die nachste Zelle aus, um zu testen:

In [22]:
# Test: Neuer Bot-Instance, gleiche Datenbank
bot2 = PersistentChatbot()

# Sollte Max's Historie kennen!
response = bot2.chat("max_session", "Was war mein Interesse nochmal?")
mprint(f"**KI (nach 'Neustart'):** {response}")

# Historie anzeigen
bot2.show_history("max_session")

Datenbank initialisiert: ./chat_memory.db


**KI (nach 'Neustart'):** Du hast gesagt, dass du dich für Machine Learning interessierst. Gibt es etwas Spezielles, das du darüber wissen möchtest?

### Thread: max_session (8 Nachrichten)

---

1. **Human:** Hallo! Ich bin Max und komme aus Munchen.

...

4. **KI:** Das ist großartig! Machine Learning ist ein spannendes und vielseitiges Feld. Gibt es einen bestimmten Bereich oder ein Thema im Machine Learning, das dich besonders interessiert? Zum Beispiel: Supervised Learning, Unsupervised Learning, neuronale Netze oder praktische Anwendungen?

5. **Human:** Woher komme ich nochmal?

6. **KI:** Du hast mir gesagt, dass du aus München kommst. Gibt es etwas Bestimmtes über München oder Machine Learning, worüber du sprechen möchtest?

7. **Human:** Was war mein Interesse nochmal?

8. **KI:** Du hast gesagt, dass du dich für Machine Learning interessierst. Gibt es etwas Spezielles, das du darüber wissen möchtest?

# A | Aufgaben
---

<p><font color='black' size="5">
Aufgabe 1: Trimming-Limit testen
</font></p>

**Schwierigkeit:** 1/5

Andern Sie `MAX_MESSAGES` auf 4 und fuhren Sie eine langere Konversation. Beobachten Sie, wann Informationen verloren gehen.

<p><font color='black' size="5">
Aufgabe 2: Summary-Qualitat verbessern
</font></p>

**Schwierigkeit:** 2/5

Verbessern Sie den `summary_prompt`, um wichtige Informationen (Namen, Orte, Praferenzen) besser zu extrahieren.

<p><font color='black' size="5">
Aufgabe 3: Interaktiver CLI-Chatbot
</font></p>

**Schwierigkeit:** 3/5

Erweitern Sie `PersistentChatbot` um eine interaktive Schleife mit Befehlen:
- `exit` - Beenden
- `history` - Historie anzeigen
- `new` - Neuen Thread starten
- `threads` - Alle Threads listen

<p><font color='black' size="5">
Aufgabe 4: Hybrid Memory (Trimming + Summary + DB)
</font></p>

**Schwierigkeit:** 4/5

Kombinieren Sie alle drei Strategien:
1. Speicherung in SQLite
2. Automatisches Trimming auf die letzten N Nachrichten
3. Zusammenfassung der alteren Nachrichten (in DB gespeichert)

# B | Datenbank auslesen
---

Dieser Abschnitt zeigt, wie die SQLite-Datenbank (`chat_memory.db`) direkt ausgelesen werden kann - nutzlich fur Debugging, Analyse oder Export.

In [23]:
import sqlite3
import os

def read_all_threads_from_db(db_path: str = DB_PATH):
    """
    Liest alle Threads und Nachrichten aus der chat_memory.db Datenbank.
    """
    if not os.path.exists(db_path):
        print(f"Fehler: Datenbankdatei '{db_path}' wurde nicht gefunden.")
        return

    mprint(f"### Lese Datenbank: {db_path}")
    mprint("---")

    with sqlite3.connect(db_path) as conn:
        # Alle Threads mit Statistiken
        threads = conn.execute("""
            SELECT thread_id, COUNT(*) as msg_count, MAX(timestamp) as last_msg
            FROM messages
            GROUP BY thread_id
            ORDER BY last_msg DESC
        """).fetchall()

        if not threads:
            print("Keine Threads in der Datenbank gefunden.")
            return

        mprint(f"**{len(threads)} Threads gefunden**\n")

        # Jeden Thread mit Nachrichten anzeigen
        for thread_id, msg_count, last_msg in threads:
            mprint(f"#### Thread: {thread_id}")
            mprint(f"*{msg_count} Nachrichten, zuletzt: {last_msg}*\n")

            # Nachrichten des Threads
            messages = conn.execute("""
                SELECT role, content, timestamp
                FROM messages
                WHERE thread_id = ?
                ORDER BY id
            """, (thread_id,)).fetchall()

            for i, (role, content, ts) in enumerate(messages, 1):
                role_display = "Human" if role == "human" else "KI"
                # Inhalt kurzen wenn zu lang
                content_short = content[:100] + "..." if len(content) > 100 else content
                mprint(f"{i}. **{role_display}:** {content_short}")

            # Zusammenfassung (falls vorhanden)
            summary = conn.execute(
                "SELECT summary FROM summaries WHERE thread_id = ?",
                (thread_id,)
            ).fetchone()

            if summary and summary[0]:
                mprint(f"\n*Zusammenfassung:* {summary[0][:150]}...")

            mprint("")  # Leerzeile zwischen Threads

In [24]:
# Alle Threads aus der Datenbank auslesen
read_all_threads_from_db()

### Lese Datenbank: ./chat_memory.db

---

**2 Threads gefunden**


#### Thread: max_session

*8 Nachrichten, zuletzt: 2025-12-03 13:59:27*


...

8. **KI:** Du hast gesagt, dass du dich für Machine Learning interessierst. Gibt es etwas Spezielles, das du da...

#### Thread: emma_session

*2 Nachrichten, zuletzt: 2025-12-03 13:59:25*


1. **Human:** Hi! Ich bin Emma aus Berlin.

2. **KI:** Hallo Emma! Schön, dich kennenzulernen. Wie kann ich dir heute helfen?

In [25]:
import json

def export_thread_to_json(thread_id: str, db_path: str = DB_PATH) -> dict:
    """Exportiert einen Thread als JSON."""
    with sqlite3.connect(db_path) as conn:
        messages = conn.execute("""
            SELECT role, content, timestamp
            FROM messages
            WHERE thread_id = ?
            ORDER BY id
        """, (thread_id,)).fetchall()

        summary = conn.execute(
            "SELECT summary FROM summaries WHERE thread_id = ?",
            (thread_id,)
        ).fetchone()

    data = {
        "thread_id": thread_id,
        "messages": [
            {"role": r, "content": c, "timestamp": t}
            for r, c, t in messages
        ],
        "summary": summary[0] if summary else None
    }

    return data

# Beispiel: Thread als JSON exportieren
thread_data = export_thread_to_json("max_session")
print(json.dumps(thread_data, indent=2, ensure_ascii=False))

{
  "thread_id": "max_session",
  "messages": [
    {
      "role": "human",
      "content": "Hallo! Ich bin Max und komme aus Munchen.",
      "timestamp": "2025-12-03 13:59:22"
    },
    {
      "role": "ai",
      "content": "Hallo Max! Schön, dich kennenzulernen. Wie kann ich dir heute helfen?",
      "timestamp": "2025-12-03 13:59:22"
    },
    {
      "role": "human",
      "content": "Ich interessiere mich fur Machine Learning.",
      "timestamp": "2025-12-03 13:59:24"
    },
    {
      "role": "ai",
      "content": "Das ist großartig! Machine Learning ist ein spannendes und vielseitiges Feld. Gibt es einen bestimmten Bereich oder ein Thema im Machine Learning, das dich besonders interessiert? Zum Beispiel: Supervised Learning, Unsupervised Learning, neuronale Netze oder praktische Anwendungen?",
      "timestamp": "2025-12-03 13:59:24"
    },
    {
      "role": "human",
      "content": "Woher komme ich nochmal?",
      "timestamp": "2025-12-03 13:59:26"
    },
    {
   

In [26]:
def delete_all_threads(db_path: str = DB_PATH):
    """Loscht alle Threads aus der Datenbank (Cleanup)."""
    with sqlite3.connect(db_path) as conn:
        conn.execute("DELETE FROM messages")
        conn.execute("DELETE FROM summaries")
        conn.commit()
    print(f"Alle Threads geloscht aus: {db_path}")

# Auskommentiert, um versehentliches Loschen zu verhindern:
# delete_all_threads()